In [3]:
from pymongo import MongoClient
from bs4 import BeautifulSoup as bs
import requests
import html as ht
import pandas as pd
from time import sleep
import re


client = MongoClient('mongodb://127.0.0.1:27017')
db = client['vacancy']
vacancydb = db.vacancy


Функция парсинга hh

In [66]:
def parse_hh(name, max_page):
    #proxies = {'http': 'http://proxy.xxx.xxx.ru:3128','https': 'http://proxy.xxx.xxx.ru:3128',}
    link = 'https://spb.hh.ru/search/vacancy?area=2&text='
    user_agent = {'User-agent': 'Mozilla/5.0'}
    result = {'vnames': [], 'salaries': [], 'employer': [], 'hrefs': []}
    
    for page in range(1, max_page): 
        url = link + name + '&page=' + str(page);
        html = requests.get(url, headers=user_agent).text
        soup = bs(html,'lxml')
        vacancy_divs = soup.select('div[data-qa="vacancy-serp__results"] div.vacancy-serp-item')
        for div in vacancy_divs:
            link_and_name = div.select('a[data-qa="vacancy-serp__vacancy-title"]')
            
            employeer= div.select('a[data-qa="vacancy-serp__vacancy-employer"]')
            salary = div.select('div[data-qa="vacancy-serp__vacancy-compensation"]')
            if len(salary) == 0:
                salary = [-1]
            else:
                salary = ht.unescape(salary[0].text).replace(u'\xa0', u'').replace(' ', '')
                if "-" in salary:
                    salary_list = re.split(r'-', salary)
                    salary_list[0] = re.findall(r'(\d+)', salary_list[0])[0]
                    salary_list[1] = re.findall(r'(\d+)', salary_list[1])[0]
                    salary = salary_list
                else:
                    salary = re.findall(r'(\d+)', salary)
            salary = [int(i) for i in salary]
            result['salaries'].append(salary)
            result['vnames'].append(link_and_name[0].text)
            result['hrefs'].append(link_and_name[0]['href'])
            result['employer'].append(employeer[0].text.replace(u'\xa0', u' '))
        
    
    result['frame'] = pd.DataFrame({'Title' : result['vnames'],
                                    'Salary' : result['salaries'],
                                    'Employer': result['employer'],
                                    'VLink' :  result['hrefs']})
    sleep(1)
    return result
        



In [67]:
max_page = 0
search_text = ''
try:
    max_page = int(input('До какой страницы парсим? '))
except Exception:
    print('Не правильно введены данные')
try:
    search_text = input('Поисковый запрос: ')
except Exception:
    print('Не правильно введены данные')

До какой страницы парсим? 10
Поисковый запрос: Программист


In [68]:
if max_page > 0 and search_text != '':
    hh_result = parse_hh(search_text, max_page)
    hh_result['frame']
else:
    print('Не правильно введены данные')

In [69]:
vac_rows =  hh_result['frame'].to_dict('records')
vac_rows

[{'Title': 'Программист 1С',
  'Salary': [120000, 120000],
  'Employer': ' СКРАП, группа компаний',
  'VLink': 'https://spb.hh.ru/vacancy/32244423?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82'},
 {'Title': 'Программист c#',
  'Salary': [75000, 125000],
  'Employer': ' Tools For Brokers',
  'VLink': 'https://spb.hh.ru/vacancy/32296880?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82'},
 {'Title': 'Php - программист Bitrix / Bitrix24',
  'Salary': [80000, 120000],
  'Employer': 'ООО Диджитал дайнемикс',
  'VLink': 'https://spb.hh.ru/vacancy/32142067?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82'},
 {'Title': 'Программист 1С',
  'Salary': [130000, 170000],
  'Employer': ' Интерлизинг',
  'VLink': 'https://spb.hh.ru/vacancy/31981231?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82'},
 {'Title': 'Программист PHP, Javascript',
  'Salary': [150000],
  'Employer': 'ООО Биомис',
  'VLink': 'ht

In [70]:
vacancydb.insert_many(vac_rows)

In [1]:
max_sal = -1;
try:
    max_sal = int(input('больше какой суммы вы хотите увидеть зарплаты? '))
except Exception:
    print('Не правильно введены данные')


больше какой суммы вы хотите увидеть зарплаты? 150000


In [4]:
vacs = list(vacancydb.find({
    "Salary": {"$gte" : max_sal}},
    {"Title": 1, "Salary": 1, "Employer": 1, "VLink": 1, "_id":0}
))
pd.set_option('display.max_colwidth', -1)
dfvacs =  pd.DataFrame(vacs, columns=['Title', 'Salary', 'Employer', 'VLink']);
dfvacs

,Title,Salary,Employer,VLink
0,Программист 1С,"[130000, 170000]",Интерлизинг,https://spb.hh.ru/vacancy/31981231?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82
1,"Программист PHP, Javascript",[150000],ООО Биомис,https://spb.hh.ru/vacancy/32355019?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82
2,Старший программист 1С,[150000],ООО Топ Трейд,https://spb.hh.ru/vacancy/32178892?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82
3,Ведущий программист PHP/PHP Developer,"[120000, 160000]",ООО Mad Wave,https://spb.hh.ru/vacancy/32039169?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82
4,"Web - программист (Битрикс, Bitrix, PHP)","[85000, 150000]",ООО АФОНЯ,https://spb.hh.ru/vacancy/31913465?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82
5,Программист / JavaScript Developer,"[90000, 150000]",ООО СЗ РЦАИ,https://spb.hh.ru/vacancy/31984951?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82
6,Разработчик / Инженер-программист (Python),[170000],NGENIX,https://spb.hh.ru/vacancy/32203300?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82
7,Программист микроконтроллеров‎ / Инженер-разработчик,[150000],Segnetics,https://spb.hh.ru/vacancy/31981347?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82
8,Программист 1С,"[50000, 150000]",ООО 33 Решения,https://spb.hh.ru/vacancy/32130501?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82
9,PL/pgSQL Developer / PostgreSQL DBA,[160000],ООО Риал Тайм Технологии,https://spb.hh.ru/vacancy/30996704?query=%D0%9F%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%81%D1%82
